# Основы программирования в Python

*Алла Тамбовцева, НИУ ВШЭ*

## Web scraping в Python: библиотеки `requests` и `BeautifulSoup`

Мы уже немного познакомились со структурой html-файлов, теперь попробуем выгрузить информацию из реальной страницы. Зайдем на [сайт](http://www.tulaoblduma.ru/) Тульской областной Думы в [раздел](http://www.tulaoblduma.ru/sostav_i_struktura/deputaty_tod/) *Депутаты Тульской областной Думы*. 

![](tula.png)

**Наша задача:** выгрузить информацию о депутатах в датафрейм *pandas*, чтобы потом сохранить все в csv-файл.

Сначала сгрузим весь html-код страницы и сохраним его в отдельную переменную. Для этого нам понадобится библиотека *requests*. 

In [1]:
import requests

Сохраним ссылку в переменную `url` для удобства и выгрузим страницу. (Разумеется, это будет работать при подключении к интернету. Если соединение будет отключено, Python выдаст *NewConnectionError*).

In [2]:
url = "http://www.tulaoblduma.ru/sostav_i_struktura/deputaty_tod/"

In [3]:
page = requests.get(url)

Если мы просто посмотрим на объект, мы ничего особенного не увидим:

In [4]:
page

<Response [200]>

Поэтому импортируем *BeautifulSoup* и скормим одноименной функции страницу в виде текста:

In [5]:
from bs4 import BeautifulSoup

In [6]:
text = page.text
soup = BeautifulSoup(text, "lxml")

Если выведем `soup` на экран, мы увидим то же самое, что в режиме разработчика или в режиме происмотра исходного кода (`view-source` через *Ctrl+U*). 

In [ ]:
soup

Для просмотра выглядит не очень удобно. "Причешем" наш `soup` ‒ воспользуемся методом `.prettify()` в сочетании с функцией `print()`.

In [ ]:
print(soup.prettify())

В такой выдаче ориентироваться гораздо удобнее (но при желании, то же можно увидеть в браузере, на большом экране). 


Теперь вернемся к нашей задаче. Видно, что информация о депутатах представлена в таблице. Следовательно, поиск нужно производить по тэгу `<table>`. Воспользуемся методом `.find_all()`:

In [ ]:
soup.find_all('table')

Метод `.find_all` возращает список с кусками html-кода, которые соответствуют указанному тэгу.
И тут мы сталкиваемся с проблемой. Несмотря на то, что визуально кажется, что таблица одна, на странице их несколько (меню сверху, содержание снизу и прочее). Найдем длину полученного выше списка:

In [7]:
len(soup.find_all('table')) # целых пять таблиц

5

Если мы посмотрим на исходный код страницы и посчитаем тэги `<table>`, мы обнаружим, что нужная нам таблица третья, то есть с индексом 2. 

In [8]:
soup.find_all('table')[2]

<table class="deputats" width="100%">
<tbody>
<thead><tr>
<td align="center" valign="center" width="5%"><b>Фото</b></td>
<td align="center" valign="center" width="20%"><b>ФИО</b></td>
<td align="center" valign="center" width="25%"><b>Фракция</b></td>
<td align="center" valign="center" width="25%"><b>Комитет</b></td>
<td align="center" valign="center" width="25%"><b>Комиссия</b></td>
</tr></thead>
<tr>
<td width="5%"><a href="deputat_info.php?117692"><img alt="" border="0" height="67" src="/upload/iblock/5a3/abakumov1.jpg" width="50"/></a></td>
<td valign="top" width="20%"><a href="deputat_info.php?117692">Абакумов Владимир Евгеньевич</a></td>
<td valign="top" width="25%">Фракция «Единая Россия»<br/>Член фракции</td>
<td valign="top" width="25%"><a href="/sostav_i_struktura/sostav_komitetov_i_komissiy/index.php#8285">Комитет по строительству, жилищно-коммунальному и дорожному хозяйству </a><br/>Член комитета</td>
<td valign="top" width="25%"><a href="/sostav_i_struktura/sostav_komitetov

Обязательно ли считать таблицы вручную? Зависит от страницы. Давайте посмотрим внимательнее на атрибуты объектов типа `table` в html-коде и проверим, можно ли поступить проще. Если сравнивать кусочки кода для разных таблиц, видно, что у таблицы с депутатами есть отличительная черта ‒ класс `deputats`:

Этой особенностью можно было воспользоваться ‒ указать атрибут в виде словаря:

In [ ]:
soup.find_all('table', {"class": "deputats"})

И считать ничего не пришлось! Сохраним результат и будем двигаться дальше. 

In [9]:
table = soup.find_all('table', {"class": "deputats"})[0] # 0 - индекс единственного элемента в списке выше

Таблицу нашли, теперь из нее нужно извлечь имена депутатов и их партийную принадлежность. Сначала разберемся с именами. По коду видно, что и имена, и партии, и комитеты с комиссиями заключены в тэги `<td>`. Можно ли найти какую-то зацепку, чтобы ловить имена? Да! Посмотрите внимательно на атрибут `valign`. У ячеек с ФИО он равен `"top"`. Этим-то мы и воспользуемся.

In [10]:
for t in table.find_all('td', valign = "top"):
    print(t.text)

Абакумов Владимир Евгеньевич
Фракция «Единая Россия»Член фракции
Комитет по строительству, жилищно-коммунальному и дорожному хозяйству Член комитета

Алёшина Галина Ивановна
Фракция «Единая Россия»Член фракции
Комитет по социальной политикеЧлен комитета
Комиссия по регламенту и депутатской этикеЧлен комиссии
Альховик Алексей Иванович
Фракция «Единая Россия»Член фракции
Комитет по социальной политикеЧлен комитета

Артемьев Сергей Александрович
Фракция «Единая Россия»Член фракции
Комитет по вопросам собственности и земельным отношениямЧлен комитета

Атанов Егор Васильевич
Фракция «Единая Россия»Член фракции
Комитет по вопросам собственности и земельным отношениямЧлен комитета

Балберов Александр Александрович
Фракция «ЛДПР»Руководитель фракции
 

Белов Сергей Александрович
Фракция «Единая Россия»Член фракции
Комитет по государственному строительству, безопасности и местному самоуправлениюЧлен комитета

Бычков Денис Владимирович
 
Комитет по вопросам собственности и земельным отношениямЧл

Что получилось? Атрибутом `valign="top"` обладают не только ячейки с именами! Давайте найдем другую зацепку, но чтобы наверняка! Обратим внимание на атрибут `width` ‒ ширина ячейки. У ФИО он равен 20%, а у других ‒ 25%! Воспользуемся этим фактом.

In [11]:
for t in table.find_all('td', valign= "top", width = "20%"):
    print(t.text)

Абакумов Владимир Евгеньевич
Алёшина Галина Ивановна
Альховик Алексей Иванович
Артемьев Сергей Александрович
Атанов Егор Васильевич
Балберов Александр Александрович
Белов Сергей Александрович
Бычков Денис Владимирович
Воробьев Николай Юрьевич
Выставкин Михаил Борисович
Грязев Михаил Васильевич
Ермаков Александр Сергеевич
Зайцева Ольга Сергеевна
Залетин Сергей Викторович
Иванцов Михаил Евгеньевич
Кирьянова Елена Сергеевна
Кондрашов Юрий Викторович
Котик Людмила Ивановна
Лебедев Алексей Александрович
Макаровец Николай Александрович
Малазония Надежда Николаевна
Марьясова Юлия Александровна
Моисеев Юрий Фясыхович
Москалец Александр Петрович
Николаева Наталия Вячеславовна
Нуждихин Григорий Вячеславович
Панин Владимир Алексеевич
Парамонова Ольга Владимировна
Попов Николай Кузьмич
Рем Александр Викторович
Самошин Андрей Анатольевич
Симонов Александр Федорович
Слюсарева Ольга Анатольевна
Судариков Анатолий Павлович
Тесов Михаил Николаевич
Толстая Екатерина Александровна
Трифонов Виктор Алексан

Получилось!

(К слову, при создании своих html-страниц очень рекомендуется задавать размер в относительном, а не абсолютном виде, то есть в процентах от общего размера страницы, а не фиксированным числом. Если выставить определенную ширину объекта, например, 10 см, то она никак не будет изменяться в зависимости от браузера, масштаба, устройства, на котором просматривается страница, что плохо.)

Будем выбирать только те ячейки, где `width` равен `25%`. Из них будем доставать только текст (ФИО) и добавлять его в заранее приготовленный пустой список `names`.

In [12]:
names = []
for t in table.find_all('td', valign= "top", width = "20%"):
    names.append(t.text)

Посмотрим на список:

In [13]:
names

['Абакумов Владимир Евгеньевич',
 'Алёшина Галина Ивановна',
 'Альховик Алексей Иванович',
 'Артемьев Сергей Александрович',
 'Атанов Егор Васильевич',
 'Балберов Александр Александрович',
 'Белов Сергей Александрович',
 'Бычков Денис Владимирович',
 'Воробьев Николай Юрьевич',
 'Выставкин Михаил Борисович',
 'Грязев Михаил Васильевич',
 'Ермаков Александр Сергеевич',
 'Зайцева Ольга Сергеевна',
 'Залетин Сергей Викторович',
 'Иванцов Михаил Евгеньевич',
 'Кирьянова Елена Сергеевна',
 'Кондрашов Юрий Викторович',
 'Котик Людмила Ивановна',
 'Лебедев Алексей Александрович',
 'Макаровец Николай Александрович',
 'Малазония Надежда Николаевна',
 'Марьясова Юлия Александровна',
 'Моисеев Юрий Фясыхович',
 'Москалец Александр Петрович',
 'Николаева Наталия Вячеславовна',
 'Нуждихин Григорий Вячеславович',
 'Панин Владимир Алексеевич',
 'Парамонова Ольга Владимировна',
 'Попов Николай Кузьмич',
 'Рем Александр Викторович',
 'Самошин Андрей Анатольевич',
 'Симонов Александр Федорович',
 'Слюса

Теперь перейдем к партиям. Тут нет никакого отличительного атрибута, но, к счастью, в каждой нужной ячейке есть слово *Фракция*. Давайте вооружимся этим фактом и с помощью цикла и условия заполним список `fracs`:

In [14]:
fracs = []
for t in table.find_all('td', valign= "top", width = "25%"):
    if "Фракция" in t.text:
        fracs.append(t.text)

Вроде бы, все хорошо:

In [15]:
fracs

['Фракция «Единая Россия»Член фракции',
 'Фракция «Единая Россия»Член фракции',
 'Фракция «Единая Россия»Член фракции',
 'Фракция «Единая Россия»Член фракции',
 'Фракция «Единая Россия»Член фракции',
 'Фракция «ЛДПР»Руководитель фракции',
 'Фракция «Единая Россия»Член фракции',
 'Фракция «Единая Россия»Член фракции',
 'Фракция «ЛДПР»Заместитель руководителя фракции',
 'Фракция «Единая Россия»Член фракции',
 'Фракция «Единая Россия»Член фракции',
 'Фракция «Единая Россия»Член фракции',
 'Фракция «Единая Россия»Член фракции',
 'Фракция «Единая Россия»Член фракции',
 'Фракция «Единая Россия»Член фракции',
 'Фракция «Единая Россия»Член фракции',
 'Фракция «Единая Россия»Член фракции',
 'Фракция «КПРФ»Руководитель фракции',
 'Фракция «Единая Россия»Член фракции',
 'Фракция «Единая Россия»Член фракции',
 'Фракция «Единая Россия»Руководитель фракции',
 'Фракция «КПРФ»Заместитель руководителя фракции',
 'Фракция «Единая Россия»Член фракции',
 'Фракция «Единая Россия»Член фракции',
 'Фракция «Е

На самом деле, не очень. Давайте сравним число депутатов и партий:

In [16]:
print(len(names))
print(len(fracs))

38
36


Партий получилось меньше! И это неудивительно: в таблице есть два беспартийных депутата. Так как их всего двое, давайте просто вручную добавим на соответствующие им позиции в списке `fracs` пустые строки. Заодно вспомним про метод `.insert()` ‒ метод, который позволяет вставить в список новый элемент на определенную позию, сдвинув остальные.

In [17]:
names.index('Бычков Денис Владимирович') # место 7

7

In [18]:
names.index('Трифонов Виктор Александрович') # место 36

36

In [19]:
fracs.insert(7, "")
fracs.insert(36, "")
fracs

['Фракция «Единая Россия»Член фракции',
 'Фракция «Единая Россия»Член фракции',
 'Фракция «Единая Россия»Член фракции',
 'Фракция «Единая Россия»Член фракции',
 'Фракция «Единая Россия»Член фракции',
 'Фракция «ЛДПР»Руководитель фракции',
 'Фракция «Единая Россия»Член фракции',
 '',
 'Фракция «Единая Россия»Член фракции',
 'Фракция «ЛДПР»Заместитель руководителя фракции',
 'Фракция «Единая Россия»Член фракции',
 'Фракция «Единая Россия»Член фракции',
 'Фракция «Единая Россия»Член фракции',
 'Фракция «Единая Россия»Член фракции',
 'Фракция «Единая Россия»Член фракции',
 'Фракция «Единая Россия»Член фракции',
 'Фракция «Единая Россия»Член фракции',
 'Фракция «Единая Россия»Член фракции',
 'Фракция «КПРФ»Руководитель фракции',
 'Фракция «Единая Россия»Член фракции',
 'Фракция «Единая Россия»Член фракции',
 'Фракция «Единая Россия»Руководитель фракции',
 'Фракция «КПРФ»Заместитель руководителя фракции',
 'Фракция «Единая Россия»Член фракции',
 'Фракция «Единая Россия»Член фракции',
 'Фракц

Теперь все хорошо. Разобраться с комитетами и комиссиями мы сегодня не успеем, но вот выцепить из элементов списка `fracs` только названия партий сумеем. И на помощь нам придут уже знакомые регулярные выражения.

Импортируем библиотеку *re*.

In [20]:
import re

Воспользуемся функцией `findall()`: найдем все названия внутри кавычек-елочек (их проще скопировать из текста или кода). 

In [21]:
parties = []
for f in fracs:
    p = re.findall('«.+»', f)[0]
    parties.append(p)

IndexError: list index out of range

Почему мы получили ошибку? Посмотрим, что представляют собой совпадения, найденные с помощью `findall()`:

In [22]:
for f in fracs:
    print(re.findall('«.+»', f))

['«Единая Россия»']
['«Единая Россия»']
['«Единая Россия»']
['«Единая Россия»']
['«Единая Россия»']
['«ЛДПР»']
['«Единая Россия»']
[]
['«Единая Россия»']
['«ЛДПР»']
['«Единая Россия»']
['«Единая Россия»']
['«Единая Россия»']
['«Единая Россия»']
['«Единая Россия»']
['«Единая Россия»']
['«Единая Россия»']
['«Единая Россия»']
['«КПРФ»']
['«Единая Россия»']
['«Единая Россия»']
['«Единая Россия»']
['«КПРФ»']
['«Единая Россия»']
['«Единая Россия»']
['«Единая Россия»']
['«Единая Россия»']
['«Единая Россия»']
['«Единая Россия»']
['«Единая Россия»']
['«Единая Россия»']
['«Единая Россия»']
['«Единая Россия»']
['«Единая Россия»']
['«Единая Россия»']
['«Единая Россия»']
[]
['«Единая Россия»']


У нас есть пустые списки! Они соответствуют беспартийным депутатам. Поэтому в таких списках без элементов Python не может найти элемент с индексом 0. Давайте добавим условную конструкцию: если длина списка не равна 0, то пусть в `parties` добавляется единственный элемент из списка, возвращенного `findall()`, если длина списка равна 0, то пусть в `parties` добавляется пустая строка.

In [23]:
parties = []
for f in fracs:
    if len(f) != 0:
        p = re.findall('«.+»', f)[0]
        parties.append(p)
    else:
        parties.append('')

In [24]:
parties

['«Единая Россия»',
 '«Единая Россия»',
 '«Единая Россия»',
 '«Единая Россия»',
 '«Единая Россия»',
 '«ЛДПР»',
 '«Единая Россия»',
 '',
 '«Единая Россия»',
 '«ЛДПР»',
 '«Единая Россия»',
 '«Единая Россия»',
 '«Единая Россия»',
 '«Единая Россия»',
 '«Единая Россия»',
 '«Единая Россия»',
 '«Единая Россия»',
 '«Единая Россия»',
 '«КПРФ»',
 '«Единая Россия»',
 '«Единая Россия»',
 '«Единая Россия»',
 '«КПРФ»',
 '«Единая Россия»',
 '«Единая Россия»',
 '«Единая Россия»',
 '«Единая Россия»',
 '«Единая Россия»',
 '«Единая Россия»',
 '«Единая Россия»',
 '«Единая Россия»',
 '«Единая Россия»',
 '«Единая Россия»',
 '«Единая Россия»',
 '«Единая Россия»',
 '«Единая Россия»',
 '',
 '«Единая Россия»']

А теперь уберем сами кавычки: если названия партий будут хранится в датафрейме в таком виде, искать их будет неудобно, поскольку придется искать или копировать откуда-то эти кавычки-елочки.
Для этого выберем в строке все символы, кроме первого (с индексом 0) и последнего (с индексом -1).

In [25]:
parties = []
for f in fracs:
    if len(f) != 0:
        p = re.findall('«.+»', f)[0]
        parties.append(p[1:-1]) # правый конец не включается в интервал
    else:
        parties.append('')

In [26]:
parties

['Единая Россия',
 'Единая Россия',
 'Единая Россия',
 'Единая Россия',
 'Единая Россия',
 'ЛДПР',
 'Единая Россия',
 '',
 'Единая Россия',
 'ЛДПР',
 'Единая Россия',
 'Единая Россия',
 'Единая Россия',
 'Единая Россия',
 'Единая Россия',
 'Единая Россия',
 'Единая Россия',
 'Единая Россия',
 'КПРФ',
 'Единая Россия',
 'Единая Россия',
 'Единая Россия',
 'КПРФ',
 'Единая Россия',
 'Единая Россия',
 'Единая Россия',
 'Единая Россия',
 'Единая Россия',
 'Единая Россия',
 'Единая Россия',
 'Единая Россия',
 'Единая Россия',
 'Единая Россия',
 'Единая Россия',
 'Единая Россия',
 'Единая Россия',
 '',
 'Единая Россия']

Теперь у нас есть два списка: ФИО депутатов и их партийная принадлежность. Осталось объединить их в датафрейм *pandas*.

Импортируем библиотеку:

In [27]:
import pandas as pd

Создадим датафрейм, подав на вход функции `DataFrame()` словарь, ключами которого являются названия столбцов, а значениями ‒ списки, которые станут столбцами таблицы.

In [28]:
df = pd.DataFrame({'name': names, 'party': parties})

Готово!

In [29]:
df.head()

,name,party
0,Абакумов Владимир Евгеньевич,Единая Россия
1,Алёшина Галина Ивановна,Единая Россия
2,Альховик Алексей Иванович,Единая Россия
3,Артемьев Сергей Александрович,Единая Россия
4,Атанов Егор Васильевич,Единая Россия


Можем выгрузить таблицу в csv-файл и на этом закончить.

In [30]:
df.to_csv("Tula_Obl_Duma.csv")